In [2]:
from csv import reader
import numpy as np
from unidecode import unidecode
import pandas as pd
import statsmodels.api as sm


#### Nesse teste de hipótese estariamos avaliando qual a influência das mulheres nas votações, será que os homens possuem além de maior representativade maior facilidade para aprovação de propostas?

In [79]:
deputados = pd.read_csv('Arquivos Limpos/deputados.csv').dropna()

In [96]:
deputados.dtypes

Unnamed: 0              int64
deputado_id             int64
nome                   object
idLegislaturaFinal      int64
nomeCivil              object
siglaSexo              object
ufNascimento           object
municipioNascimento    object
dtype: object

In [ ]:
deputados = pd.read_csv('Arquivos Limpos/deputados.csv').dropna()

In [81]:
votacoes = pd.read_csv('Arquivos Limpos/votacoes.csv').dropna()

In [52]:
votacoes.dtypes

Unnamed: 0                                     int64
id                                            object
idOrgao                                        int64
idEvento                                       int64
aprovacao                                    float64
votosSim                                       int64
votosNao                                       int64
votosOutros                                    int64
ultimaApresentacaoProposicao_idProposicao      int64
dtype: object

In [82]:
votos = pd.read_csv('Arquivos Limpos/votacoesVotos.csv').dropna()

In [83]:
votos.dtypes

Unnamed: 0                int64
idVotacao                object
voto                      int64
deputado_id               int64
deputado_siglaPartido    object
pardido_id                int64
deputado_siglaUf         object
dtype: object

In [84]:
data_dep_prop = pd.merge(deputados, proposicoes, left_on='deputado_id', right_on='idDeputadoAutor')

In [85]:
data_voto_vota = pd.merge(votacoes,votos, left_on='id', right_on='idVotacao')

In [99]:
 data= pd.merge(data_dep_prop,votacoes, left_on='idProposicao', right_on='ultimaApresentacaoProposicao_idProposicao')

In [100]:
data_dep_prop.head()


,Unnamed: 0_x,deputado_id,nome,idLegislaturaFinal,nomeCivil,siglaSexo,ufNascimento,municipioNascimento,Unnamed: 0_y,idProposicao,idDeputadoAutor
0,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,1553,2233724,66179.0
1,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,3528,2231834,66179.0
2,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,3529,2231833,66179.0
3,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,3614,2231756,66179.0
4,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,12419,2224878,66179.0


In [101]:
data.head()

,Unnamed: 0_x,deputado_id,nome,idLegislaturaFinal,nomeCivil,siglaSexo,ufNascimento,municipioNascimento,Unnamed: 0_y,idProposicao,idDeputadoAutor,Unnamed: 0,id,idOrgao,idEvento,aprovacao,votosSim,votosNao,votosOutros,ultimaApresentacaoProposicao_idProposicao
0,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,58184,2214248,66179.0,4769,2200728-26,537870,56863,1.0,0,0,0,2214248
1,2842,66179,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitoria,115951,2200839,66179.0,2142,2187314-25,2006,55575,1.0,0,0,0,2200839
2,2845,66828,Fausto Pinato,56,FAUSTO RUY PINATO,M,SP,Fernandopolis,2407,2232899,66828.0,8439,2218910-21,2001,58868,1.0,0,0,0,2232899
3,2845,66828,Fausto Pinato,56,FAUSTO RUY PINATO,M,SP,Fernandopolis,58179,2214253,66828.0,5506,2204001-26,2001,57249,1.0,0,0,0,2214253
4,2845,66828,Fausto Pinato,56,FAUSTO RUY PINATO,M,SP,Fernandopolis,110109,2204198,66828.0,3559,2190989-25,2001,56277,1.0,0,0,0,2204198


In [102]:
data1= data.idProposicao.value_counts().head(5)

In [91]:
data2= data.aprovacao.value_counts().head(5)

In [114]:
values = data['siglaSexo'].unique()

for value in values[:-1]:
    data[value] = [1 if v == value else 0 for v in data['siglaSexo']]

In [115]:
data[values[:-1]]

,F
0,1
1,1
2,0
3,0
4,0
...,...
3064,0
3065,0
3066,0
3067,0


In [116]:
model = sm.Logit(data['aprovacao'], data[values[:-1]])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.610393
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              aprovacao   No. Observations:                 3069
Model:                          Logit   Df Residuals:                     3068
Method:                           MLE   Df Model:                            0
Date:                Mon, 26 Oct 2020   Pseudo R-squ.:                  -1.691
Time:                        18:58:03   Log-Likelihood:                -1873.3
converged:                       True   LL-Null:                       -696.18
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
F              3.2871      0.247     13.307      0.000       2.803       3.771
==============================================================================
"""

In [118]:
data1= pd.merge(data_dep_prop,data_voto_vota, left_on='idProposicao', right_on='ultimaApresentacaoProposicao_idProposicao')

In [119]:
data1

,Unnamed: 0_x_x,deputado_id_x,nome,idLegislaturaFinal,nomeCivil,siglaSexo,ufNascimento,municipioNascimento,Unnamed: 0_y_x,idProposicao,...,votosNao,votosOutros,ultimaApresentacaoProposicao_idProposicao,Unnamed: 0_y_y,idVotacao,voto,deputado_id_y,deputado_siglaPartido,pardido_id,deputado_siglaUf
0,3156,73788,Ricardo Barros,56,RICARDO JOSE MAGALHAES BARROS,M,PR,Maringa,4603,2230755,...,268,4,2230755,8266,2075448-116,-1,204521,PSL,36837,SP
1,3156,73788,Ricardo Barros,56,RICARDO JOSE MAGALHAES BARROS,M,PR,Maringa,4603,2230755,...,268,4,2230755,8267,2075448-116,-1,204379,PROS,36763,AP
2,3156,73788,Ricardo Barros,56,RICARDO JOSE MAGALHAES BARROS,M,PR,Maringa,4603,2230755,...,268,4,2230755,8268,2075448-116,-1,204528,NOVO,37901,SP
3,3156,73788,Ricardo Barros,56,RICARDO JOSE MAGALHAES BARROS,M,PR,Maringa,4603,2230755,...,268,4,2230755,8269,2075448-116,-1,121948,PP,37903,GO
4,3156,73788,Ricardo Barros,56,RICARDO JOSE MAGALHAES BARROS,M,PR,Maringa,4603,2230755,...,268,4,2230755,8270,2075448-116,-1,160508,PT,36844,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52165,7612,207309,Eneias Reis,56,ENEIAS JOSE DOS REIS,M,MG,Coronel Fabriciano,125740,2192385,...,65,54,2192385,3074,1672576-109,-1,74439,PDT,36786,PE
52166,7612,207309,Eneias Reis,56,ENEIAS JOSE DOS REIS,M,MG,Coronel Fabriciano,125740,2192385,...,65,54,2192385,3075,1672576-109,0,178889,PT,36844,MA
52167,7612,207309,Eneias Reis,56,ENEIAS JOSE DOS REIS,M,MG,Coronel Fabriciano,125740,2192385,...,65,54,2192385,3076,1672576-109,0,204559,PT,36844,BA
52168,7612,207309,Eneias Reis,56,ENEIAS JOSE DOS REIS,M,MG,Coronel Fabriciano,125740,2192385,...,65,54,2192385,3077,1672576-109,-1,160632,SOLIDARIEDADE,37904,MG


In [121]:
values = data1['deputado_siglaPartido'].unique()

for value in values[:-1]:
    data1[value] = [1 if v == value else 0 for v in data1['deputado_siglaPartido']]